In [1]:
# 2V25saC0pDFc8heT7wxsjigW3CF_32oQPdVZZP9wSMGd29FCa
# 2NqtkyBMF0KF99ofEU1fGz1pCJS_3XehxpoU47JGBiqY6tV7M

!pip install gradio_client paddleocr paddlepaddle paddlepaddle-gpu deepface transformers TTS openai-whisper pyngrok fastapi[all] nest_asyncio -q
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok authtoken 2V25saC0pDFc8heT7wxsjigW3CF_32oQPdVZZP9wSMGd29FCa
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb
!sudo apt update && sudo apt upgrade && sudo apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 6.6 MB/s eta 0:00:00
deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 3s (131 kB/s)
Reading pac

In [2]:
import whisper
from fastapi import FastAPI, UploadFile, File, HTTPException, Body
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uuid
import os
import uvicorn
import torch
from TTS.api import TTS
from fastapi.responses import FileResponse
import asyncio
import gc
from transformers import AutoTokenizer, AutoModelForTokenClassification
from deepface import DeepFace
import paddleocr
from gradio_client import Client

device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'

client = Client("https://huggingfaceh4-falcon-chat.hf.space/", serialize=False)
ocr_reader = paddleocr.PaddleOCR(lang="hi",use_gpu=True)
voice_model = whisper.load_model("large-v2").to(device)
tts = TTS('tts_models/en/jenny/jenny').to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")
quantized_ner_model = torch.ao.quantization.quantize_dynamic(model,{torch.nn.Linear},dtype=torch.qint8)
del model
port=8888
gc.collect()

/usr/local/lib/python3.10/dist-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):


Loaded as API: https://huggingfaceh4-falcon-chat.hf.space/ ✔
[2023/09/07 18:26:06] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/devanagari/devanagari_PP-OCRv4_rec_infer', rec_image_inverse=True, 

3711

In [4]:
app = FastAPI(title="SIH 2023 Backend")

app.add_middleware(
CORSMiddleware,
allow_origins=["*"],
allow_credentials=True,
allow_methods=["*"],
allow_headers=["*"],
)

class TTSNER(BaseModel):
    text: str
    emotion: str = "Cheerful & Professional"

class FaceResponse(BaseModel):
    prediction: bool

class FaceDetect(BaseModel):
    prediction: str

def remove(known_image_path,test_image_path):
    os.remove(known_image_path)
    os.remove(test_image_path)

@app.get("/")
async def home():
    return "SIH - LICHT DEN CODE"

@app.post("/face-detect/")
async def face_detect(img: UploadFile = File(...)):
    img_path = f"{uuid.uuid4()}.jpg"
    with open(img_path, "wb") as known_image_file:
        known_image_file.write(img.file.read())
    result = DeepFace.extract_faces(img_path,enforce_detection=False)[0].get('confidence')
    os.remove(img_path)
    if result>5:
        return FaceDetect(prediction="Done!!!")
    else:
        return FaceDetect(prediction="Face could not be detected. Please confirm that the picture is a face photo.")

@app.post("/face-match/")
async def face_match(known_face: UploadFile = File(...), test_face: UploadFile = File(...)):
    try:
        known_image_path = f"{uuid.uuid4()}.jpg"
        test_image_path = f"{uuid.uuid4()}.jpg"
        with open(known_image_path, "wb") as known_image_file:
            known_image_file.write(known_face.file.read())
        with open(test_image_path, "wb") as test_image_file:
            test_image_file.write(test_face.file.read())
        result = DeepFace.verify(known_image_path, test_image_path, model_name='Facenet512', distance_metric='euclidean_l2').get('verified')
        remove(known_image_path,test_image_path)
        return FaceResponse(prediction=result)

    except Exception as e:
        remove(known_image_path,test_image_path)
        if str(e)=='''Face could not be detected. Please confirm that the picture is a face photo or consider to set enforce_detection param to False.''':
            raise HTTPException(status_code=399, detail="Face could not be detected. Please confirm that the picture is a face photo.")
        else:
            raise HTTPException(status_code=500, detail=str(e))

@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):
    audio_path = f"{uuid.uuid4()}.webm"
    with open(audio_path, "wb") as f:
        f.write(await file.read())
    result = voice_model.transcribe(whisper.pad_or_trim(whisper.load_audio(audio_path)))["text"]
    os.remove(audio_path)
    return {"text": result}

@app.post("/ocr/")
async def OCR(file: UploadFile = File(...)):
    pic_path = f"{uuid.uuid4()}.jpg"
    with open(pic_path, "wb") as f:
        f.write(await file.read())
    result = ' '.join([word[1][0] for line in ocr_reader.ocr(pic_path) for word in line])
    os.remove(pic_path)
    return {"text": result}

@app.post("/ner/")
async def get_ner_endpoint(request: TTSNER = Body(...)):
    sentence = request.text.strip()
    tok_sentence = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        logits = quantized_ner_model(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            quantized_ner_model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

    return {"LOC": [word for word, label in zip(sentence.split(' '), predicted_labels) if "LOC" in label]}

@app.post("/coqui-tts/")
async def coqui_tts(request: TTSNER = Body(...)):
    out = f"{uuid.uuid4()}.ogg"
    async def remove():
        loop = asyncio.get_event_loop()
        await loop.run_in_executor(None, lambda: os.remove(out))
    tts.tts_to_file(request.text, file_path=out, emotion=request.emotion)
    return FileResponse(out,headers={"Content-Disposition":f"attachment; filename={out}"},background=remove)

@app.post("/chat/")
async def falcon(request: TTSNER = Body(...)):
  result = client.predict(
  request.text, # str in 'Type an input and press Enter' Textbox component
  "", # str (filepath to JSON file) in 'parameter_9' Chatbot component
  "The following is a conversation between a highly knowledgeable and intelligent AI assistant, called Falcon, and a human user, called User. In the following interactions, User and Falcon will converse in natural language, and Falcon will answer User's questions. Falcon was built to be respectful, polite and inclusive. Falcon was built by the Technology Innovation Institute in Abu Dhabi. Falcon will never decline to answer a question, and always attempts to give an answer that User would be satisfied with. It knows a lot, and always tells the truth. The conversation begins.", # str in 'Instructions' Textbox component
  0.7, # int | float (numeric value between 0.1 and 2.0) in 'Temperature' Slider component
  0.8, # int | float (numeric value between 0.1 and 0.99) in 'p (nucleus sampling)' Slider component
  fn_index=1)[0][1]
  return {"text":result}

if __name__=="__main__":
    ngrok_tunnel = ngrok.connect(port)
    print('Public URL:', ngrok_tunnel.public_url)
    nest_asyncio.apply()
    uvicorn.run(app,port=port)

Public URL: https://75b0-34-125-191-35.ngrok-free.app


INFO:     Started server process [21471]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8888 (Press CTRL+C to quit)


INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "POST /chat/ HTTP/1.1" 200 OK
INFO:     2405:201:35:eb5d:bd5c:2069:542e:7f87:0 - "GET / HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /coqui-tts/ HTTP/1.1" 200 OK
 > Text splitted to sentences.
['I want to go from Matunga Station to Borivali Station']
 > Processing time: 8.840007543563843
 > Real-time factor: 2.5149381347265556
INFO:     49.36.101.103:0 - "OPTIONS /ner/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /ner/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /ner/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /ner/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /ner/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /ner/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /ner/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 426, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 292, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist

INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "GET / HTTP/1.1" 200 OK
INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2405:201:35:eb5d:4a53:af0c:3b57:b372:0 - "POST /chat/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "POST /chat/ HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 426, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 292, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist

INFO:     49.36.101.103:0 - "POST /chat/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "POST /chat/ HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 426, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 292, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist

INFO:     49.36.101.103:0 - "POST /chat/ HTTP/1.1" 200 OK
INFO:     171.51.200.159:0 - "POST /transcribe/ HTTP/1.1" 200 OK
 > Text splitted to sentences.
['I want to go from the other station to Boreveli station right now.']
 > Processing time: 0.6121945381164551
 > Real-time factor: 0.15658484221582106
INFO:     171.51.200.159:0 - "POST /coqui-tts/ HTTP/1.1" 200 OK
INFO:     171.51.200.159:0 - "POST /chat/ HTTP/1.1" 200 OK
INFO:     171.51.200.159:0 - "POST /ner/ HTTP/1.1" 200 OK
 > Text splitted to sentences.
['How do i go from Matunga to borivali station']
 > Processing time: 0.6053035259246826
 > Real-time factor: 0.1987425388830084
INFO:     49.36.101.103:0 - "POST /coqui-tts/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "POST /chat/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /chat/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "OPTIONS /coqui-tts/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "POST /chat/ HTTP/1.1" 200 OK
INFO:     49.36.101.103:0 - "POST /coqui-tts/ HTT

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [21471]
